In [ ]:
'''
Notebook for working as a demo in the API deployment to add new features and/or fix bugs

Most of the developments were done in the renderClass.py and then I was using this notebook for testing. 

NOTE: You need to use the example_client (or example_client_copy in my case).
This is the client from the repository of the annotation service (https://github.com/gpan12/mv-annotation-service).
Panagiotis Galopoulos can give you access so that you can clone it and import the client. 
'''

In [3]:
import requests
import tempfile
from pytorch3d.io import load_obj, load_ply
import sys
sys.path.append("..")
import os
from pathlib import Path
import json
from rendererClass import RendererClass
import itertools
from PIL import Image
import numpy as np
import cv2
sys.path.append("/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/Annotation_API/mv-annotation-service")
import example_client_copy
import matplotlib
import matplotlib.pyplot as plt
import shutil
import time
from loguru import logger
from PIL import Image
import io
import trimesh
import zipfile


# Main pipeline

In [8]:
'''
Function to test the API pipeline
It can work with URL and local files
URL is giving error when it comes to large gdrive files (because permission is needed)
Therefore we make it work with local files as well for testing purposes (this is not part of the annotation API)

'''




def multiview_API_pipeline(obj_local_path, #path were the object is stored
                           download_link, #url IF use_url true
                           download_format, #format that the resuest has
                           use_img # If tru use image directly
                           ):

    params = {
            "image_size": 1024,
            "camera_dist": [2],  
            "elevation": [0],
            "azim_angle": [0,45,90,135,180,225,270,315,360],
            "filename": "",
            "file_extension": "glb",
            "z_coord": 0, 
            "save_img":False,                        
            "save_path": ""
            }

    asset_type = params["file_extension"]

    with tempfile.TemporaryDirectory() as tmpdir:

        # The context manager will automatically delete this directory after this section
        logger.info(f"Created a temporary directory: {tmpdir}")


        params["save_path"] = tmpdir


        if download_format == "url": #works with just one obj
            print(f'Working with url, download from {download_link}')
            obj = requests.get(download_link).content
            
        #reads the obj from local in order to get it in a similar format as requests.get()
        elif download_format == "local": 
        
            print(f'Working with local path, load from {obj_local_path}')
            with open(obj_local_path, "rb") as object:
                obj = object.read()
            
                temp_path =  os.path.normpath(obj_local_path)

                src_dir = os.sep + os.path.join(*temp_path.split(os.sep)[:-1])
                shutil.copytree(src_dir, tmpdir,dirs_exist_ok = True)
        else:
            raise ValueError("Need to chose between url, local or zip")
            
        
        annotations = []

        
        if asset_type == "obj":
            with open(os.path.join(tmpdir,"object." + asset_type),"wb") as f:
                f.write(obj)
            
                params["filename"] = f.name
        #NOTE: very immature implementation
        #TODO: update the condition with more efficient code
        elif asset_type == "zip":
            
            
            #---this is not needed but we want to work as the input in the service---
            '''
            destination_path = os.path.join(tmpdir,"temp_zip.zip")
            print(f"Zip folder to {destination_path}")
        
            #NOTE: change this!
            make_archive(os.path.dirname(obj_local_path), destination_path)
            '''
            # -------------------------------------------------------------------
            
            with zipfile.ZipFile(io.BytesIO(obj), 'r') as zip_ref:
                zip_ref.extractall(tmpdir)
                
            for root, dirs, files in os.walk(tmpdir):
                for file in files:
                    
                    _, file_ext = os.path.splitext(os.path.join(root,file))

                    if file_ext == ".obj":
                        
                        logger.info("I am in the zip dir")
                        
                        params["filename"] = os.path.join(root,file)
                        params["file_extension"] = "obj"
                        
        Renderer  = RendererClass(params)

        logger.info(params)
        Renderer.create_mesh_object(obj)
            
            
        #load the pars to render 3D
        all_dist = params["camera_dist"]
        all_elev = params["elevation"]
        all_azim = params["azim_angle"]

        #get all combinations
        all_combs =  list(itertools.product(*[all_dist,all_elev,all_azim]))


        logger.info(f'Number of images to annotate: {len(all_combs)}')

        


        #for loop to go through all the provided multiview images
        for comb in all_combs:
            
            logger.info(f'Working with combination:{comb}')
            
            
            
            #store the image
            img = Renderer.render_image(comb[0],comb[1],comb[2])
            
            path_to_store = os.path.join(tmpdir,str(comb)+".png")
              
                
            logger.info(path_to_store)
            matplotlib.image.imsave(path_to_store, img)    
            
            #return img
            
            
            #define the asset path of the png
            asset_path = path_to_store
            #print(f'Asset path: {asset_path}')
            logger.info(f'Asset path: {asset_path}')
                 
            
            if use_img:
            
                logger.info(f'Work with img directly:')
                
                #convert numpy image to bytes
                PIL_im = Image.fromarray((img * 255).astype(np.uint8))
                img_byte_arr = io.BytesIO()
                PIL_im.save(img_byte_arr, format='PNG')
                img_byte_arr = img_byte_arr.getvalue()
                
                #return img,img_byte_arr
                # Use the annotation client
                try:
                    resp = client.sync_annotate(
                        asset_url=None,
                        asset_path=None,
                        asset_bytes = img_byte_arr,
                        models = [
                            example_client_copy.AnnotationModel.OBJECT_DETECTION,
                        ],
                        metadata={"id": "1"},
                    )
                    object_detection_result, error = resp.get_result(
                        example_client_copy.AnnotationModel.OBJECT_DETECTION
                    )

                    if object_detection_result:
                        
                        # check is the specific model result is ok
                        logger.info(object_detection_result)
                        
                except example_client_copy.AnnotationError:
                    # Something pretty bad happened and all responses are lost.
                    # This error should not happen under normal circumstances.
                    # It's probably a bug that needs to be reported.
                    logger.info("An error occurred")
        
        
        
            else:
                # Use the annotation client
                try:
                    resp = client.sync_annotate(
                        asset_url=None,
                        asset_path=asset_path,
                        models = [
                            example_client_copy.AnnotationModel.OBJECT_DETECTION,
                        ],
                        metadata={"id": "1"},
                    )
                    object_detection_result, error = resp.get_result(
                        example_client_copy.AnnotationModel.OBJECT_DETECTION
                    )

                    if object_detection_result:
                        
                        # check is the specific model result is ok
                        logger.info(object_detection_result)
                        
                except example_client_copy.AnnotationError:
                    # Something pretty bad happened and all responses are lost.
                    # This error should not happen under normal circumstances.
                    # It's probably a bug that needs to be reported.
                    logger.info("An error occurred")
            
                
            
            #append the annotation list with new detections
            [annotations.append(annot.object) for annot in object_detection_result.detections]
            
            logger.info(os.path.isdir(tmpdir))
            
    logger.info(os.path.isdir(tmpdir))       
    logger.info(set(annotations))

    return set(annotations)



In [5]:

#  initialize client
client = example_client_copy.MVAnnotationClient(
        address="160.40.53.61:37527",
        secure=False,
    )


In [14]:

#obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/Police_Officer/UrbanPoliceOfficer.obj"
#obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/Astronaut/Astronaut.glb"
#obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/Apple/apples.gltf"
# obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/OFF_format/B.off"
# obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/OFF_format/7.off"
#obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/Liberty/Liberty_v4.glb"
obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/working_room/Darbo.obj"
#obj_local_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/data/random_example/Apple/apples.obj"

download_prefix = 'https://docs.google.com/uc?export=download&id='
use_img = True
download_format = "url"
params_path = "/home/andstasi/Projects/MediaVerse/3D_to_2D_converter/pytorch3d-renderer/params_inference.json"

link = "https://drive.google.com/file/d/1MUdFiMGkjHi3IYWjaxDE8tLzNbgonBSF/view?usp=sharing"
link = "https://drive.google.com/file/d/147WSr-W7ouf2vwoo-ZglFoE2R4qlf2Tp/view?usp=sharing"
#link = "https://drive.google.com/drive/folders/1KwdSAzF097T5szuP8BAli0T7CZjFWrT0?usp=sharing"
link_astronaut = "https://drive.google.com/file/d/1RwjwTdgGbeJR2U3VY6cx5DuEX0cjOFJn/view?usp=sharing"
link_astronaut_zip = "https://drive.google.com/file/d/14KyaZUhKvGK4kvQtnx_4A-dzUUtoD_Zx/view?usp=sharing"
link_police_man = "https://drive.google.com/file/d/1iNS4r9w4k8ekprVX1wAMYvB2_g-UdSLV/view?usp=sharing"
off_format = "https://drive.google.com/file/d/1unNCZPLl0-nClFQgVHefKjFC1e3HymCW/view?usp=sharing"
link_apple_gltf = "https://drive.google.com/file/d/1Ik4oi5nJGPnUDbgqyUX9qYOLPBFRvBD_/view?usp=sharing"
link_apple_obj = "https://drive.google.com/file/d/1Y7Rr9rtKep4LdC3IS90-rJX9xgptTQao/view?usp=sharing"
linke_apple_zip = "https://drive.google.com/file/d/1F7I7adr9o22CLUhYg3AuG6IrHdO59bSJ/view?usp=sharing"
link_tiger_ply = "https://drive.google.com/file/d/1Vcsv0Rqym4LmOYqs1M97dXAgQu5p-jeR/view?usp=sharing"
link_single_bed_ply = "https://drive.google.com/file/d/1moqu2Iy8HwTdNBW7XSYqgWeR2Iu8YqQl/view?usp=sharing"
link_single_bed_stl = "https://drive.google.com/file/d/1WWLtRnWZnqyXo6cKN0pn0KVy-jQKzScd/view?usp=sharing"
link_single_bed_obj = "https://drive.google.com/file/d/1KiCkOYxlESa9gJRe9BLO_SH9iFr5HRqL/view?usp=sharing"
link_full_body_ply = "https://drive.google.com/file/d/1MjhKP-2P4ZYxh9WP1gwBGSFHRiCkUJXT/view?usp=sharing"
link_chair_obj = "https://drive.google.com/file/d/1YTocHeQ-BX9dDh8q2qh6hQAxUUyMUlw6/view?usp=sharing"
link_char_ply = "https://drive.google.com/file/d/1htM8gPxbxP1qRB0s_Pa3qcZMzYXpgkVE/view?usp=sharing"
link_apple_off = "https://drive.google.com/file/d/1RNxgOiwJRmAfBeFu4-YzmRE_HEa0VZ3q/view?usp=sharing"
link_wm_glb = "https://drive.google.com/file/d/1ulLCEFgA0jFskJC_l5XiKdDh404P2Slh/view?usp=sharing"
link_armchair_glb = "https://drive.google.com/file/d/179DvBpKyQvvA46VsF9MxTVwY293NJXY_/view?usp=sharing"
link_sofa_glb = "https://drive.google.com/file/d/1tSQAeIyOhu6rf1ulkbU6w0loI3CnVRUy/view?usp=sharing"
link_airplane_glb = "https://drive.google.com/file/d/186N_o-eGlcK_qn2Q293MGbhCUDeXuKxq/view?usp=sharing"
link_kichennete_gltf = "https://drive.google.com/file/d/1SfWFbqz60hCvThQa1hVwrrRkz_Cz2fDe/view?usp=sharing"
link_wooden_chair_glb = "https://drive.google.com/file/d/1WAmlgEwVT9LSWarISOUhfQQUGpI6qITZ/view?usp=sharing"
link_mcclaren_glb = "https://drive.google.com/file/d/1nhMtMI0rpHetFuFFP0Cx5H7GXzO8T97G/view?usp=sharing"
link_new_person_glb = "https://drive.google.com/file/d/1Fb_DNg2Ps8AFxLfxXjZfL0jYhm2St1aI/view?usp=sharing"
download_link = download_prefix+link_police_man.strip().split("/")[-2]
#download_link = download_prefix+link.strip().split("/")[-1]

print(download_link)

#img,img_byte_arr = multiview_API_pipeline(obj_local_path,download_link,use_url,use_img)




https://docs.google.com/uc?export=download&id=1iNS4r9w4k8ekprVX1wAMYvB2_g-UdSLV


In [13]:
multiview_API_pipeline(obj_local_path, #path were the object is stored
                           download_link, #url IF use_url true
                           download_format, #format that the resuest has
                           use_img # If tru use image directly
                           )

2022-03-10 10:33:40.022 | INFO     | __main__:multiview_API_pipeline:35 - Created a temporary directory: /tmp/tmptdad6aeb


Working with url, download from https://docs.google.com/uc?export=download&id=1Fb_DNg2Ps8AFxLfxXjZfL0jYhm2St1aI


2022-03-10 10:33:46.711 | INFO     | __main__:multiview_API_pipeline:100 - {'image_size': 1024, 'camera_dist': [2], 'elevation': [0], 'azim_angle': [0, 45, 90, 135, 180, 225, 270, 315, 360], 'filename': '', 'file_extension': 'glb', 'z_coord': 0, 'save_img': False, 'save_path': '/tmp/tmptdad6aeb'}
2022-03-10 10:33:46.714 | INFO     | rendererClass:create_mesh_object:185 - Working with not OBJ format
2022-03-10 10:33:46.736 | INFO     | rendererClass:create_mesh_object:198 - Created a temporary directory: /tmp/tmpbiw7afj9
/home/andstasi/anaconda3/envs/triton_inf/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2022-03-10 10:33:51.266 | INFO     | rendererClass:create_mesh_object:231 - Working wit

normalize the input in a unit shpere


2022-03-10 10:33:53.575 | INFO     | __main__:multiview_API_pipeline:172 - success: 1
detections {
  object: "person"
  confidence: 0.9993463158607483
  bbox {
    top_left_x: 0.12812581658363342
    top_left_y: 0.3508036136627197
    width: 0.9674507975578308
    height: 0.6549289226531982
  }
}

2022-03-10 10:33:53.577 | INFO     | __main__:multiview_API_pipeline:213 - True
2022-03-10 10:33:53.578 | INFO     | __main__:multiview_API_pipeline:121 - Working with combination:(2, 0, 45)
2022-03-10 10:33:53.648 | INFO     | __main__:multiview_API_pipeline:131 - /tmp/tmptdad6aeb/(2, 0, 45).png
2022-03-10 10:33:53.721 | INFO     | __main__:multiview_API_pipeline:140 - Asset path: /tmp/tmptdad6aeb/(2, 0, 45).png
2022-03-10 10:33:53.721 | INFO     | __main__:multiview_API_pipeline:145 - Work with img directly:


...

2022-03-10 10:34:01.903 | INFO     | __main__:multiview_API_pipeline:172 - success: 1
detections {
  object: "person"
  confidence: 0.9987704157829285
  bbox {
    top_left_x: 0.13354113698005676
    top_left_y: 0.36973655223846436
    width: 0.9250699281692505
    height: 0.6983095407485962
  }
}

2022-03-10 10:34:01.904 | INFO     | __main__:multiview_API_pipeline:213 - True
2022-03-10 10:34:01.906 | INFO     | __main__:multiview_API_pipeline:121 - Working with combination:(2, 0, 360)
2022-03-10 10:34:01.973 | INFO     | __main__:multiview_API_pipeline:131 - /tmp/tmptdad6aeb/(2, 0, 360).png
2022-03-10 10:34:02.052 | INFO     | __main__:multiview_API_pipeline:140 - Asset path: /tmp/tmptdad6aeb/(2, 0, 360).png
2022-03-10 10:34:02.053 | INFO     | __main__:multiview_API_pipeline:145 - Work with img directly:


success: 1
detections {
  object: "person"
  confidence: 0.9987704157829285
  bbox {
    top_left_x: 0.13354113698005676
    top_left_y: 0.36973655223846436
    width: 0.9250699281692505
    height: 0.6983095407485962
  }
}



2022-03-10 10:34:03.090 | INFO     | __main__:multiview_API_pipeline:172 - success: 1
detections {
  object: "person"
  confidence: 0.9993464350700378
  bbox {
    top_left_x: 0.128122478723526
    top_left_y: 0.35080283880233765
    width: 0.9674482345581055
    height: 0.6549304127693176
  }
}

2022-03-10 10:34:03.091 | INFO     | __main__:multiview_API_pipeline:213 - True
2022-03-10 10:34:03.094 | INFO     | __main__:multiview_API_pipeline:215 - False
2022-03-10 10:34:03.096 | INFO     | __main__:multiview_API_pipeline:216 - {'person'}


success: 1
detections {
  object: "person"
  confidence: 0.9993464350700378
  bbox {
    top_left_x: 0.128122478723526
    top_left_y: 0.35080283880233765
    width: 0.9674482345581055
    height: 0.6549304127693176
  }
}



{'person'}